In [2]:
pip install tika

     |████████████████████████████████| 133kB 11.8MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.26.3
    Uninstalling urllib3-1.26.3:
      Successfully uninstalled urllib3-1.26.3


In [3]:
pip install boto3

In [4]:
from bs4 import BeautifulSoup
from tika import parser
import urllib.request
import datetime
import requests
import boto3

# AWS 인증 상수
AWS_ACCESS_KEY_ID = ''
AWS_SECRET_ACCESS_KEY = ''

# 텍스트 파일 업로드 위해 s3 리소스 사용
s3 = boto3.resource(
    's3',
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)


def extract_target():
    """
    추출 대상이 되는 타겟 리포트 목록 추출
    :return:
    """

    # 추출 일자를 현재일로 부터 일주일 이전을 시작으로 현재일자 까지를 추출 범위로 선택
    begin_date = datetime.datetime.now().date() + datetime.timedelta(days=-7)
    end_date = datetime.datetime.now().date().isoformat()
    extract_target_range_url = "https://finance.naver.com/research/company_list.nhn?keyword=&brokerCode=&searchType=writeDate&writeFromDate={}&writeToDate={}&page={}"

    _target_list = list()
    for idx in range(1, 30):
        # 1 페이지부터 30페이지까지 하드코딩으로 돌립니다
        # ToDo 페이지네이션 마지막 페이지까지 for문 돌도록 수정 필요 합니다 !
        html = requests.get(extract_target_range_url.format(begin_date, end_date, idx))
        soup = BeautifulSoup(html.text, 'html.parser')

        table_class = soup.select('table.type_1')
        trs = table_class[0].find_all('tr')

        # 딕셔너리에 사명: pdf 다운로드 링크 맵핑하여 삽입
        for tr_item in trs[2:-2]:
            tds = tr_item.find_all('td')
            if not tds[0].get('class'):
                company_td = tds[0].text.strip()
                file_path_td = tds[3].next.attrs.get('href')

                target_elem = {
                    'company': company_td,
                    'filepath': file_path_td
                }
                _target_list.append(target_elem)

    # 중복된 요소 제거 위함
    # ToDo 시간 많이 걸린다면 해당 코드 리팩토링 부탁드립니다 :)
    result_list = list({each_target['filepath']: each_target for each_target in _target_list}.values())
    return result_list


def download_file(download_url, filename):
    """
    로컬에 pdf 파일 다운로드
    :param download_url:
    :param filename:
    :return:
    """
    # ToDo 다운로드 하지 않고 읽어올 수 있다면 해당 방법으로 변경 부탁드려요
    response = urllib.request.urlopen(download_url)
    file = open(filename + ".pdf", 'wb')
    file.write(response.read())
    file.close()


def pre_processing(content):
    """
    데이터 전처리
    :param content:
    :return:
    """
    # ToDo 전처리는 아예 안되어있습니다 ~!
    return content.replace('\n', '')


target_list = extract_target()

folder_name = 'emotion' + '/' + datetime.datetime.today().date().isoformat() + '_' + datetime.datetime.today().strftime("%A")

for item in target_list:
    # ToDo .. 조금 더 깔끔하고 로지컬하게.. 부탁드려요 :( ㅎ
    download_file(item.get('filepath'), item.get('company'))
    parsed = parser.from_file(item.get('company') + ".pdf")

    upload_file_name = item.get('company') + '_result_' + '.txt'
    txt = open(upload_file_name, 'w', encoding='utf-8')
    # output.txt에 pdf파일 내용을 write
    processed = pre_processing(parsed['content'])
    print(processed, file=txt)  # ToDo 파일 with.. open 등의 쓰기 처리 방식 변경 확인 필
    txt.close()
    # txt 확장자 파일로 s3에 업로드 합니다.
    s3.meta.client.upload_file(
        upload_file_name,
        'analysis-target-data',
        folder_name + '/' + upload_file_name,
        ExtraArgs={
            'ACL': 'public-read'
        }
    )

In [2]:
from google.colab import drive # 패키지 불러오기 
from os.path import join  

ROOT = "/content/drive"     # 드라이브 기본 경로
print(ROOT)                 # print content of ROOT (Optional)
drive.mount(ROOT)           # 드라이브 기본 경로 Mount

/content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
MY_GOOGLE_DRIVE_PATH = 'My Drive/Colab Notebooks/NLP/' # 프로젝트 경로
PROJECT_PATH = join(ROOT, MY_GOOGLE_DRIVE_PATH) # 프로젝트 경로
print(PROJECT_PATH)

/content/drive/My Drive/Colab Notebooks/NLP/


In [4]:
%cd "{PROJECT_PATH}"

/content/drive/My Drive/Colab Notebooks/NLP


In [5]:
import pandas as pd
review_df = pd.read_csv("data/labeledTrainData.tsv", header = 0, sep="\t", quoting = 3)
review_df.head()


,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [6]:
review_df.shape

(25000, 3)

In [7]:
import re

# <br> html 태그는 replace 함수를 공백으로 변환
review_df['review'] = review_df['review'].str.replace('<br />', ' ')

# 파이썬의 정규 표현식 모듈인 re를 이용해 영어 문자열이 아닌 문자는 모두 공백으로 변환 
review_df['review'] = review_df['review'].apply(lambda x : re.sub("[^a-zA-Z]", " ", x))

#[^a-zA-Z]의 의미는 영어 대/소문자가 아닌 모든 문자를 찾는 것임

In [8]:
from sklearn.model_selection import train_test_split

class_Y = review_df['sentiment']
feature_X = review_df.drop(['id', 'sentiment'], axis = 1, inplace=False)
print(feature_X)

X_train, X_test, y_train, y_test = train_test_split(feature_X, class_Y, test_size = 0.3, random_state = 156)

X_train.shape, X_test.shape

                                                  review
0       With all this stuff going down at the moment ...
1         The Classic War of the Worlds   by Timothy ...
2       The film starts with a manager  Nicholas Bell...
3       It must be assumed that those who praised thi...
4       Superbly trashy and wondrously unpretentious ...
...                                                  ...
24995   It seems like more consideration has gone int...
24996   I don t believe they made this film  Complete...
24997   Guy is a loser  Can t get girls  needs to bui...
24998   This    minute documentary Bu uel made in the...
24999   I saw this movie as a child and it broke my h...

[25000 rows x 1 columns]


((17500, 1), (7500, 1))

In [9]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

In [13]:
# 스톱 워드는 English, Filtering, Ngram은 (1, 2)로 설정하여 CountVectorization 수행
# LogisticRegression의 C는 10으로 설정
pipeline = Pipeline([
                     ('cnt_vect', CountVectorizer(stop_words = 'english', ngram_range=(1, 2))), 
                     ('lr_clf', LogisticRegression(C=10))])

# Pipeline 객체를 이용해 fit(), predict()로 학습/예측 수행, predict_proba()는 roc_auc 때문에 수행. 
pipeline.fit(X_train['review'], y_train)
pred = pipeline.predict(X_test['review'])
pred_probs = pipeline.predict_proba(X_test['review'])[:, 1]

print('예측 정확도는 {0:.4f}, ROC-AUC는 {1:.4f}'.format(accuracy_score(y_test, pred), roc_auc_score(y_test, pred_probs)))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


예측 정확도는 0.8860, ROC-AUC는 0.9503


In [11]:
from xgboost import XGBClassifier

pipeline1 = Pipeline([
                     ('cnt_vect', CountVectorizer(stop_words = 'english', ngram_range=(1, 2))), 
                     ('lr_clf', XGBClassifier())])

# Pipeline 객체를 이용해 fit(),
# predict()로 학습/예측 수행, 
# predict_proba()는 roc_auc 때문에 수행. 
pipeline1.fit(X_train['review'], y_train)
pred = pipeline1.predict(X_test['review'])

In [12]:
from sklearn.metrics import accuracy_score
print('검증 정확도: ', accuracy_score(y_test, pred))
from sklearn.metrics import classification_report

print(classification_report(pred, y_test))

검증 정확도:  0.8088
              precision    recall  f1-score   support

           0       0.75      0.84      0.79      3264
           1       0.87      0.78      0.82      4236

    accuracy                           0.81      7500
   macro avg       0.81      0.81      0.81      7500
weighted avg       0.82      0.81      0.81      7500



In [17]:
!pip install eli5

     |████████████████████████████████| 112kB 11.5MB/s 


In [18]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import eli5
from eli5.sklearn import PermutationImportance

In [20]:
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 144